# Is Tokyo Right About How To Use Escalators?

Inspired by the article [Tokyo Wants People to Stand on Both Sides of the Escalator](https://www.citylab.com/transportation/2018/12/people-stand-walk-escalator-left-right-save-time-efficient/578479/), let's examine two different ways of using an escalator that can accomodate two people per step.

The first is people always stand, which is what railway officials in Tokyo want people to do. The second is people who want to stand stick to one side, leaving the other side free for those who wish to walk, which is how they're typically been used.

We'll look at several different metrics - the average amount of time people wait to actually get on the escalator, the number of people processed per second and the average amount of time people people spend in total waiting for and actually on the escalator.

In [4]:
-- Horizontal length of escalator in meters
hle = 10

-- Depth of step in meters
d = 0.40

-- Walking speed in meters per second
w = 84/60

-- Horizontal escalator speed in meters per second
s = 42/60

-- Deterministic service time in seconds
dst = s/d

-- Average people per second
y = 1/2

-- Arrival in units of fixed service time
b = y*dst

## Standing Only

We first examine the case where people simply stand. This is equivalent to a single queue with two servers and deterministic service time $D$, which is the amount of time it takes for an escalator step to appear.

The analysis of this case is difficult, but there is a nice closed-form with respect to the Lambert W function.

Let $z_i$, $z_i \neq 1$, be the $c-1$ roots of the equation $1-z^c e^{\lambda D (1-z)}=0$.

Then we have $$\mathbf{E}(n_s;c>1) = \sum_{i=1}^{c-1} \frac{1}{1-z_i} + \frac{1}{2} \left(\lambda D + \frac{c}{c-\lambda D} - c\right).$$

The expected waiting time $T_D$ in the (queue, not escalator) system is then by [Little's law](https://en.wikipedia.org/wiki/Little%27s_law) $$T_D = \frac{\mathbf{E}(n_s)}{\lambda}.$$

For our purposes we want the time spent in the queue, which is $t_Q = T_D-D$, as well as the total time spent in the queue and on the escalator, which is $T_S = t_Q + L/s$, where $L$ is the horizontal length of the escalator and $s$ is the horizontal speed of the escalator.

The number of people processed per second is the arrival rate adjusted for the time spent in the queue, which is $$\frac{1}{\frac{1}{\lambda}+t} = \frac{\lambda}{1+\lambda t}.$$

In [19]:
-- M/D/2

import Numeric.GSL.Special(lambert_W0)

-- c servers; 2 for this problem
c = 2
p = b/c

-- I believe this is always true
z_1 = -1/p*lambert_W0(p/exp(p))

-- Expected queue waiting time
eqwt = 1/(1-z_1) + 1/2*(c*p+1/(1-p)-c)
t = eqwt/y-dst

putStrLn $ "M/D/2 expected waiting time = " ++ show t
putStrLn $ "M/D/2 expected system time = " ++ show (t+hle/s)

-- People per second
ps_2 = 1/(1/y+t)

putStrLn $ "M/D/2 people per second = " ++ show ps_2

M/D/2 expected waiting time = 0.22262562150255216

M/D/2 expected system time = 14.508339907216838

M/D/2 people per second = 0.4499183264719023

## One Side Walks, One Side Stands

The analysis of a walk/stand split queue is simpler, as we have two [M/D/1](https://en.wikipedia.org/wiki/M/D/1_queue) queues with very simple closed forms for the expected waiting time.

Assuming the fraction of people that choose to walk/stand is fixed, we have two separate Poisson processes for the arrivals, which in turn means we have two independent M/D/1 queues. The time spent waiting in the queue for such a system is $$T_Q = \frac{\lambda D}{2}\cdot \frac{D}{1-\lambda D}.$$ Note that the above collapses to this as $c\to 1$. The time spent in the queue and on the escalator is then $$T_E = t_Q + \frac{L}{s}$$ for those standing and $$T_E = t_Q + \frac{L}{w+s}$$ for those walking, where $w$ is the walking speed.

The overall average waiting time in the queue is then the weighted average of the two queues, as is the expected time spent in the queue and on the escalator. The number of people processed per second is simply the sum of the number of people processed per second for each queue individually.

In [23]:
-- Walk/stand splits

f_w = 0.20
f_s = 1-f_w

y_w = f_w*y
y_s = f_s*y

p_w = y_w*dst
p_s = y_s*dst

t_w = p_w/2*dst/(1-p_w)
t_s = p_s/2*dst/(1-p_s)

q_t = f_w*t_w+f_s*t_s
putStrLn $ "Split queue expected waiting time = " ++ show q_t

s_t = f_w*(t_w+hle/(s+w)) + f_s*(t_s+hle/s)

putStrLn $ "Split queue expected system time = " ++ show s_t

-- People per second
ps_s = 1/(1/y_w+t_w) + 1/(1/y_s+t_s)

putStrLn $ "Split queue people per second = " ++ show ps_s

putStrLn ""
putStrLn $ "Ratio of M/D/2 pps to split queue pps = " ++ show (ps_2/ps_s)

Line 1: Use camelCase
Found:
s_t = ...
Why not:
sT = ...Line 1: Use camelCase
Found:
ps_s = ...
Why not:
psS = ...

Split queue expected waiting time = 1.670454545454545

Split queue expected system time = 14.051406926406928

Split queue people per second = 0.31836124748806727

Ratio of M/D/2 pps to split queue pps = 1.4132320752662146

## Conclusion

It should be noted that expected queue waiting time will always be minimized with a standing only (M/D/2) approach, as will the number people processed per second. However, under some very reasonable assumptions it's very likely that the expected amount of time spent waiting for and actually on the escalator will be smaller with the walk/stand (dual M/D/1) approach, as the walkers will spend far less time actually on the escalator. Of course, there may be other reasons to prefer people to stand only, e.g. safety. However, this suggests instead designing escalators so that they're easier and safer to climb rather than requiring everyone stand.